In [1]:
import time
import os
import json
import csv
from datetime import datetime
from datetime import date
from dateutil.relativedelta import relativedelta
from tqdm import tqdm_notebook

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [3]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC 
from selenium.webdriver.common.by import By

In [4]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common import exceptions
from collections import defaultdict


In [5]:
def get_keywordUrl(keyword):
    """ ()
    Args:
        keyword(string): 검색할 키워드
    Returns:
        url(string): keyword가 포함된 url
    """
    url = "https://search.shopping.naver.com/search/all?frm=NVSHMDL&origQuery=" + keyword +"&pagingIndex=1&pagingSize=40&productSet=model&query=" + keyword + "&sort=rel&timestamp=&viewType=list"
    return url
    
    

In [6]:
def get_detailUrl(url):
    """
    Args:
        url(string): keyword가 포함된 url
    Returns:
        url_list(list): 각 상품의 상세페이지 링크를 담은 리스트
    """
    
    options = webdriver.ChromeOptions()	
    options.add_argument('headless')	
    options.add_argument('disable-gpu')	
    driver = webdriver.Chrome('chromedriver.exe', options=options)
    driver.get(url)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(3)

    url_list = driver.find_elements_by_tag_name("div.basicList_title__3P9Q7 > a")
    url_list = [url.get_attribute('href') for url in url_list]
    return url_list

In [64]:
def get_prdInfo(url):
    """
    Args:
        url(string): 각 상품의 상세page 
    Returns:
        info_list(list): 상품이름, 가격, 제조사, 브랜드명, 등록일, 찜(좋아요) 갯수,
    """
    
    req = requests.get(url)
    cont = req.content
    soup = BeautifulSoup(cont, 'html.parser')
    
    prdInfo_list = list()
    
    # 제조사, 브랜드 모두 있는 경우
    try:
        # 제목
        title = soup.select('div > div.h_area > h2')
        title = title[0].get_text().replace(' ', "").replace("\n",'')
        # 평점
        try:
            score = soup.select('div > div.h_area > div > div.gpa')[0].text
        except:
            score = "NULL"
        # 제조사
        try:
            manufacter = soup.select(' div.summary_info._itemSection > div > div.goods_info > div > span:nth-of-type(1) > em')[0].text
        except:
            manufacter = "NULL"
        # 브랜드
        try: 
            brand = soup.select('div.summary_info._itemSection > div > div.goods_info > div > span:nth-of-type(2) > em')[0].text
        except:
            brand = "NULL"
        # 등록일
        try: 
            reg_date = soup.select('div.summary_info._itemSection > div > div.goods_info > div > span:nth-of-type(3) > em')[0].text
        except:
            reg_date = "NULL"
        # 좋아요 수
        try:
            num_like = soup.select('#jjim > em.cnt._keepCount')[0].text
        except:
            num_like = "NULL"
        # 가격
        try:
            price = soup.select('div.summary_cet > div.price_area > span > em')[0].text
        except:
            price = "NULL"
    except:
        # 제목
        title = soup.select('div > div.h_area > h2')
        title = title[0].get_text().replace(' ', "").replace("\n",'')
        # 평점
        try:
            score = soup.select('div > div.h_area > div > div.gpa')[0].text
        except:
            scroe = "NULL"
        # 제조사
        manufacter = "NULL"
        # 브랜드
        try:
            brand = soup.select(' div.summary_info._itemSection > div > div.goods_info > div > span:nth-of-type(1) > em')[0].text
        except:
            brand = "NULL"
        # 등록일
        try:
            reg_date = soup.select('div.summary_info._itemSection > div > div.goods_info > div > span:nth-of-type(2) > em')[0].text
        except:
            reg_date = "NULL"
        # 좋아요 수
        try:
            num_like = soup.select('#jjim > em.cnt._keepCount')[0].text
        except:
            num_like = "NULL"
        # 가격
        try: 
            price = soup.select('div.summary_cet > div.price_area > span > em')[0].text
        except:
            price = "NULL"
    prdInfo_list.append(title)
    prdInfo_list.append(score)
    prdInfo_list.append(manufacter)
    prdInfo_list.append(brand)
    prdInfo_list.append(reg_date)
    prdInfo_list.append(num_like)
    prdInfo_list.append(price)
    
    return prdInfo_list
    

In [65]:
def get_prdRvw(url):
    """
    Args:
        url(string): 각 상품의 상세pager
    Returns:
        reviews_list(list): 제조사, 브랜드명, 등록일, 별점, 리뷰 제목, 리뷰 본문, 이미지url
    """
    
    options = webdriver.ChromeOptions()	
    #options.add_argument('headless')	
    options.add_argument('disable-gpu')	
    driver = webdriver.Chrome('chromedriver.exe', options=options)
    driver.get(url)
    time.sleep(3)
    test = driver.find_elements_by_tag_name('#_review_list > li')

    review_list = list()

    # 첫페이지임을 나타내는 매개 변수 first
    first = True

    if first == True:
        first = False
        print('1 ~ 10page')
        page = 2

        while (page <= 11):

            test = driver.find_elements_by_tag_name('#_review_list > li')

            for cnt in test:
                temp_review_list = list()

                score = cnt.text.split("\n")[1].split(' ')[0]
                brand = cnt.text.split("\n")[1].split(' ')[1]
                cus_id = cnt.text.split("\n")[1].split(' ')[2]
                try:
                    review_date = str(20)+cnt.text.split("\n")[1].split(' ')[3]
                except Exception as e:
                    print(e)
                    print(str(cnt.text.split("\n")[1].split(' ')))
                    review_date = str(cnt.text.split("\n")[1].split(' '))
                review_title = cnt.find_element_by_tag_name('div > p').text
                review_text = cnt.find_element_by_tag_name('div > div.atc').text

                # 썸네일
                # 1. 사진
                try: 
                    thumb = cnt.find_element_by_tag_name('div.thmb > span > img').get_attribute('src')
                # 2. 영상
                except:
                    try:
                        thumb = cnt.find_element_by_tag_name('div.thmb > button > img').get_attribute('src')
                # 3. 없는 경우
                    except:
                        thumb = "NULL"

                test = driver.find_elements_by_tag_name('#_review_list > li')        
                temp_review_list.append(score)
                temp_review_list.append(brand)
                temp_review_list.append(cus_id)
                temp_review_list.append(review_date)
                temp_review_list.append(review_title)
                temp_review_list.append(review_text)
                temp_review_list.append(thumb)
                review_list.append(temp_review_list)

            try:
                print('#_review_paging > a:nth-child(%s)완료' %str(page))
                page += 1
                next_page = driver.find_element_by_tag_name('#_review_paging > a:nth-child(%s)' %str(page))
                next_page.click()
            except:
                print("리뷰 마지막 페이지")
                print('#_review_paging > a:nth-child(%s)' %str(page))
                break
            time.sleep(5)



    if first == False:
        print('11page ~ ')
        page = 4

        while (page <= 13):
            test = driver.find_elements_by_tag_name('#_review_list > li')
            for cnt in test:
                temp_review_list = list()

                score = cnt.text.split("\n")[1].split(' ')[0]
                brand = cnt.text.split("\n")[1].split(' ')[1]
                cus_id = cnt.text.split("\n")[1].split(' ')[2]
                try:
                    review_date = str(20)+cnt.text.split("\n")[1].split(' ')[3]
                except Exception as e:
                    print(e)
                    print(str(cnt.text.split("\n")[1].split(' ')))
                    review_date = str(cnt.text.split("\n")[1].split(' '))
                review_title = cnt.find_element_by_tag_name('div > p').text
                review_text = cnt.find_element_by_tag_name('div > div.atc').text

                # 썸네일
                # 1. 사진
                try: 
                    thumb = cnt.find_element_by_tag_name('div.thmb > span > img').get_attribute('src')
                # 2. 영상
                except:
                    try:
                        thumb = cnt.find_element_by_tag_name('div.thmb > button > img').get_attribute('src')
                # 3. 없는 경우
                    except:
                        thumb = "NULL"

                test = driver.find_elements_by_tag_name('#_review_list > li')        
                temp_review_list.append(score)
                temp_review_list.append(brand)
                temp_review_list.append(cus_id)
                temp_review_list.append(review_date)
                temp_review_list.append(review_title)
                temp_review_list.append(review_text)
                temp_review_list.append(thumb)
                review_list.append(temp_review_list)

            try:
                print('#_review_paging > a:nth-child(%s) 완료' %str(page))
                page += 1
                if page == 14:
                    page = 4
                    print('10page 지남')
                next_page = driver.find_element_by_tag_name('#_review_paging > a:nth-child(%s)' %str(page))
                next_page.click()
            except:
                print("리뷰 마지막 페이지")
                print('#_review_paging > a:nth-child(%s)' %str(page))
                break
            time.sleep(5)

    return review_list
    

In [66]:
def write_product_csv(keyword, product_name, product_info_list):
    """ (string, string, list) -> None
    Args:
        filename(string): 검색 키워드
        product_name(string): 제품 이름
        product_info_list(list): 제품 정보
    Retrun:
        None
        csv파일에 제품 정보 작성
    info_list(list): 상품이름, 가격, 제조사, 브랜드명, 등록일, 찜(좋아요) 갯수,
    """
    filename = keyword + '_' + product_name + '.csv'
    csv_file = open(filename, "w", encoding = 'utf-8-sig', newline= '')
    wr = csv.writer(csv_file)
    wr.writerow(['상품 이름', '가격', '제조사', '브랜드명', '등록일', '찜(좋아요) 갯수'])
    wr.writerow(product_info_list)
    csv_file.close()

In [67]:
def write_review_csv(keyword, product_name, review_lists):
    """ (string, list) -> None
    Args:
        filename(string): 파일 이름
        product_name(string): 제품 이름
        review_list(list in list): 리뷰 정보
        # reviews_lists(list): 상품 이름, 가격, 제조사, 브랜드명, 등록일, 별점, 리뷰 제목, 리뷰 본문, 이미지url
    Retrun:
        None
        csv파일에 리뷰 리스트 t 작성
    """
    filename = keyword + '_' + product_name + '_review.csv'
    csv_file = open(filename, "w", encoding = 'utf-8-sig', newline= '')
    wr = csv.writer(csv_file)
    wr.writerow(['평점', '판매점', '사용자id', '등록일', '리뷰 제목', '리뷰 본문', '이미지url'])
    
    for review in review_lists:
        wr.writerow(review)
    csv_file.close()

In [68]:
def main():
    keyword = input("검색할 키워드를 입력하세요: ")
    url = get_keywordUrl(keyword)
    url_list = get_detailUrl(url)

    print('각 상품에 대한 url 수집 완료')

    # 상품 정보
    prdInfo_list = list()
    # 리뷰 정보
    rvw_list = list()

    filename = keyword + '_' + str(date.today()) + '.csv'
    product_csv_file = open(filename, "a", encoding = 'utf-8-sig', newline= '')
    wr = csv.writer(product_csv_file)
    wr.writerow(['상품 이름', '평점', '제조사', '브랜드명', '등록일', '찜(좋아요) 갯수', '가격'])

    for url in url_list:
        temp_prdInfo_list = get_prdInfo(url)
        prdInfo_list.append(temp_prdInfo_list)

        temp_rvw_list = get_prdRvw(url)
        rvw_list.append(temp_rvw_list)
        print(url, '하나의 상품 정보 및 이에 대한 리뷰 수집 완료')
    print('전체 상품 정보 및 이에 대한 리뷰 수집 완료')

    # 상품정보 파일 작성 (list)
    for prdInfo in prdInfo_list:
        wr.writerow(prdInfo)
    print('상품정보 파일 저장완료')
    product_csv_file.close()


    # 리뷰정보 파일 작성
    for idx, reviews in enumerate(rvw_list):
        filename = keyword + '_' + str(prdInfo_list[idx][0]) + str(date.today()) + '.csv'
        review_csv_file = open(filename, "a", encoding = 'utf-8-sig', newline= '')
        wr = csv.writer(review_csv_file)
        wr.writerow(['평점', '브랜드명', '고객 ID', '등록일', '제목', '본문', '이미지 url'])    

        for review in reviews:
            wr.writerow(review)
        review_csv_file.close()
        print(prdInfo_list[idx][0], '리뷰정보 파일 저장완료')

In [69]:
%%time
if __name__ == "__main__":
    main()

검색할 키워드를 입력하세요: 닭가슴살 도시락
각 상품에 대한 url 수집 완료
1 ~ 10page
#_review_paging > a:nth-child(2)완료
#_review_paging > a:nth-child(3)완료
#_review_paging > a:nth-child(4)완료
#_review_paging > a:nth-child(5)완료
#_review_paging > a:nth-child(6)완료
#_review_paging > a:nth-child(7)완료
#_review_paging > a:nth-child(8)완료
#_review_paging > a:nth-child(9)완료
리뷰 마지막 페이지
#_review_paging > a:nth-child(10)
11page ~ 
#_review_paging > a:nth-child(4) 완료
#_review_paging > a:nth-child(5) 완료
#_review_paging > a:nth-child(6) 완료
#_review_paging > a:nth-child(7) 완료
#_review_paging > a:nth-child(8) 완료
#_review_paging > a:nth-child(9) 완료
리뷰 마지막 페이지
#_review_paging > a:nth-child(10)
https://cr.shopping.naver.com/adcr.nhn?x=rmFl%2BMapNWNmR7KeWOuZPv%2F%2F%2Fw%3D%3Ds0R3Ia1UiDv2OtsgsCPfo23zQzGXcGyb70bn%2BEY1Xju5URgVbdo%2F4PXOwP%2BCkYSb%2BIvsF232LdCw5Yjw4baJ6SX2r3vnFRDfSmoDh0T%2BvrxO3pN8NyBiuX4y1Th%2BEilkM1mgv6LYsXsAgHoZR%2BA3YTxNJOHa%2FJawJg2%2FDJ%2Bws2dWAbCY81f975eS5nfR25FBitmbKT6fiobaO7w0fEN2NzhTtRLWgUgHVCqjbH6kiokWC%2B7e6juVAl

https://cr.shopping.naver.com/adcr.nhn?x=MlutToIdhsaC27U0Gtax5v%2F%2F%2Fw%3D%3DsFqwix049tYnnh8weM7dCjbKJJ%2BBFjEkr9GyudmYpal9URgVbdo%2F4PXOwP%2BCkYSb%2BIvsF232LdCw5Yjw4baJ6SckQipiXU8cPs14%2BWjqmgIy3pN8NyBiuX4y1Th%2BEilkM1mgv6LYsXsAgHoZR%2BA3YTxNJOHa%2FJawJg2%2FDJ%2Bws2dWAbCY81f975eS5nfR25FBitmbKT6fiobaO7w0fEN2NzmJGvUKRtLjcaptWMenqhqbWCsIExFV0diS6GEcDQHNxM6uQ0%2FEuxe9Kx7BiV%2BSF2Vw2EzC1R7jC%2F43hDec%2FYLmc%2BzySEcyja6yI2wIa7nRnca58VC9yAmQyC04nezu%2B%2BWz8zbWMtJTFaAZuTYJYj51YO4Am6plsaXHSO650j1yFP2tfKoQ0NQPyiad34AQ6K9%2FdVvwtcOUlbNrGLU3RUnoO5cUrOVqhEIRauvchImcpai83EVbMDvzV06wNeZeBjZ%2Bca9tTOMn220t4i9iLe3tYK%2FNUN442dEkfO%2FtkLjKcnD38MxfZLVnMXa3E7TvClDP%2F6iiIn7QzI%2BXIqwB6FMvMcgwNALQ4lDpEqf1%2FGEUUhamfVl2hWFu5rqbLeGo%2FLLXjoDrdxPCKm7cKEATA5to3WOvmXRjNqlwVnzGXNyutm5qLRT9MoMXtcpS0wZvXShCQTUjyvDTLnnCzaXjR4TFxtplKINvTBNAF4tVLGvVEiA3%2FU2AzguC84AnoS7Lq%2BWPDzS7X5yyi%2BZ2TjDsG%2FKYx6aTo4wQuUxAQeRPWwZ%2B55Hv1T%2FoDPeAT2ZD%2BWCfnsQ%3D%3D&nvMid=23231616490&catId=50006199 하나의 상품 정보 및 이에 대한 리뷰 수집 완료

#_review_paging > a:nth-child(2)완료
리뷰 마지막 페이지
#_review_paging > a:nth-child(3)
11page ~ 
#_review_paging > a:nth-child(4) 완료
리뷰 마지막 페이지
#_review_paging > a:nth-child(5)
https://cr.shopping.naver.com/adcr.nhn?x=SaR7Gu9GzVem5T9y4Zcl3P%2F%2F%2Fw%3D%3DssRvo%2FJG3NACaKV7obZrbA8d3aEBBNSKyYnQvJr8fkCCUIoCF7tpuZrJYKKlwciwnbreuM68p6FG%2Bu%2Fa3frwitLa3HZPJdMGYPPbBUmHr07LiebtUXDqZ76NUxILd%2BNc1YOI5TwAxea9fQAf%2BJHWy80ls1M9xucXxkPPt7QTmSRFr6kdrp7BvT8OlwZ7P37YytWL5Ea0XbCuEY8Khhs43VPkMZ7uHCMj68CNg%2Bw9XlojIpFjo%2FeZfQ7piG6BSrk6lgyyQp%2BnAmN8%2FwAdGMuW0klzxYiawkLppbZR2fOQd9RYDRS3Hu8BnUwWcYTRWAAihBiEvgB645yFkDOSg06D0tA08Sw0YW63cc41Azs%2FDE3Vlc6P1YajnJs%2FPQ4MWP%2BA7wc%2B%2FjkR2vl%2B%2B%2B%2F%2BVPNlrhYgmqU0ZfI%2Fx5rGg8tmeVByokV1OQYx9OrQ5KqECsnAPSo0adg%2FaZAYXCXaj0Al5E1c9gv6AqHYNDzSgyBRAIa%2BZdv1%2FPmu1G9qAj5cxTIsbC0r%2BUmM4rOUlXYgQF%2BmvEd8KzgSmVweziZE5PcTrTOasG%2BgyflqNHe6Mo5ZmFBIwNjr0OpDwWipDrVKo%2FExMQ5Wp0mPW3%2FFCYJS729z9%2FqmUiXZUvrchO3nnvS%2FQghA8nyI%2B%2Fr0QMg1092QLiXPRJW5K7O1msmMvR1g%2Fc%2BP6b%2

https://cr.shopping.naver.com/adcr.nhn?x=c2NK94mqM1ArRReTVkKT0f%2F%2F%2Fw%3D%3DsPiZ9o9SD%2BGwGwD9rV9e5jO%2Bc1acUTVafXC154WOJ3iVURgVbdo%2F4PXOwP%2BCkYSb%2BIvsF232LdCw5Yjw4baJ6Sf7VPIVWk14GJl8ASF4kAB63pN8NyBiuX4y1Th%2BEilkM1mgv6LYsXsAgHoZR%2BA3YTxNJOHa%2FJawJg2%2FDJ%2Bws2dWAbCY81f975eS5nfR25FBipriVTNUFNXA4oBwrJtENpPxO8gomEql3xgFbNDAt6S0NkSYysyx8TGfJE6MQI8V56xD57hFIbQb%2FklB900yo%2BloDjcQBDY1F%2BKzX2ZnBz%2BArdF411kcE99Rr6HERw6m9t5CiGfC2CLp%2BEuKfGzFA7wF7isUVsp3M6Ct9mT6vB4JWEXeXwSaubPu0qUaxQfPPa%2BpHa6ewb0%2FDpcGez9%2B2Mmaf7oVVn%2BbTDsruUdhAA7ss1Gg2dZ%2BQLyCfCXp5e2CB1EL7PBh6vo%2Bzr1Db8r4ssTOZvREhpqqrGGHV4Tf%2FuETH4mDSyI6ZBQRKoR%2FwUleHBHS7iYdd4BTrgASNQmm8sBfosZcv1svKtcwDMQlwLKPlR9rut1Zqe4rHMZX511gAY2NFcUaAGlqM4nu3LYZy8PiORyduGASa9LsFUtlzH7aGMEszX6S%2Fz31uEZN94ozW5nDJXloeCLRsDEfaBTUhVyUhCmzOzMYYJMOYRJxj6ecSzxwVRNV%2FYRemZK5z1sCEwNOB7I4JHPHshGaMcRfQZwX1YTm%2Bnmh%2FHug2KfsAb7vNjqh%2FZemCCOboDhnweAmG&nvMid=21255052849&catId=50001878 하나의 상품 정보 및 이에 대한 리뷰 수집 완료
1 ~ 10page
#_review_paging > a:nth-

#_review_paging > a:nth-child(3)
11page ~ 
#_review_paging > a:nth-child(4) 완료
리뷰 마지막 페이지
#_review_paging > a:nth-child(5)
https://cr.shopping.naver.com/adcr.nhn?x=3dukBHCPum%2BQIQQwnvyZhP%2F%2F%2Fw%3D%3DsAs9mlEf6QsiunqJUtLw%2FwNoFGJUghNxlbB7oqONgCJ5URgVbdo%2F4PXOwP%2BCkYSb%2BIvsF232LdCw5Yjw4baJ6SdrvIl2b%2FsyvKbtJIy3G1j23pN8NyBiuX4y1Th%2BEilkM1mgv6LYsXsAgHoZR%2BA3YTxNJOHa%2FJawJg2%2FDJ%2Bws2dWAbCY81f975eS5nfR25FBipriVTNUFNXA4oBwrJtENpDmDGpL9gWIJERDPNZvnIn3v0%2B2WjdveBdiCo6g9IjHe6xD57hFIbQb%2FklB900yo%2BloDjcQBDY1F%2BKzX2ZnBz%2BArdF411kcE99Rr6HERw6m9t5CiGfC2CLp%2BEuKfGzFA7wF7isUVsp3M6Ct9mT6vB4JWEXeXwSaubPu0qUaxQfPPa%2BpHa6ewb0%2FDpcGez9%2B2Mmaf7oVVn%2BbTDsruUdhAA7shTiDFmW1Vx%2FN6MYfcVLo51EL7PBh6vo%2Bzr1Db8r4sseZ7Q%2FJ8sO%2FTY6OK%2BDwjnKH6K137PlTKQpiMVW1UPKx6BHS7iYdd4BTrgASNQmm8sJljgKldTxeHSmQ0VUJCQgWv2qZtTmS6sEl4nXtAn02gdB9ZvMPP1%2Bx9pNRBy4vFsaxvmt4aYpKh60kFNvaRvMoJnZLxDyyazNtYAUpXm7ZqKhE%2Fi1bOLwTOYtHK%2BS%2BkwobW8sZgq%2BgWpqwfbrIoPK0RpOFzCbaM6OU97uoPifjr01C0TybfuoVjuDY7TqE54dcI8bs9S2a

https://cr.shopping.naver.com/adcr.nhn?x=QomuTLvXfWS%2F6dcizPcGf%2F%2F%2F%2Fw%3D%3DskhsLSjAaHyQZBzsfpd%2FtVSyz8bhfKQ107fWwKtcZVaFURgVbdo%2F4PXOwP%2BCkYSb%2BIvsF232LdCw5Yjw4baJ6SeSGPeGubRquxglAjHfwt4i3pN8NyBiuX4y1Th%2BEilkM1mgv6LYsXsAgHoZR%2BA3YTxNJOHa%2FJawJg2%2FDJ%2Bws2dWAbCY81f975eS5nfR25FBipriVTNUFNXA4oBwrJtENpFHWkgf22lWA46YqbV8PCRtIcxTEYmZ45%2FeetSAAYgAY6xD57hFIbQb%2FklB900yo%2BloDjcQBDY1F%2BKzX2ZnBz%2BArdF411kcE99Rr6HERw6m9t5CiGfC2CLp%2BEuKfGzFA7wF7isUVsp3M6Ct9mT6vB4JWEXeXwSaubPu0qUaxQfPPa%2BpHa6ewb0%2FDpcGez9%2B2Mmaf7oVVn%2BbTDsruUdhAA7u7Kt%2BivQwNn6DMtTnkO7kE1EL7PBh6vo%2Bzr1Db8r4ssUKJiXtCddFoQ5cxnUJr5katWhr7hP72xaErnXg63ZFOBHS7iYdd4BTrgASNQmm8sFC3fqoCiWfU5vwodsImxfwP2qiuPJYyjlyHtnjNeWJsQd4hIGbWj%2FH%2FHFfYDDruLFTzxomkq7Vomu8UWU4nEdFfBjXFRn2vAS1sfpst8c2rNNwAyUQrgrpw2vJkr6o9%2BlRWIuOtnkQvrtn3CYK3Z3rCfHYIPCsubXfDQ1zobHhH71tqtWQablf%2F77SlIExh57JvGjK4VGmSl9eHfzULuEnn7gmph8z2zvtMgA8K0VQbOhyWUIIyPFUpWztUfPMydg1QmAmnoJRdq4K9btjkCM0%3D&nvMid=22137976581&catId=50001878 하나의 상품 정보 및 이에 대한 

리뷰 마지막 페이지
#_review_paging > a:nth-child(3)
11page ~ 
#_review_paging > a:nth-child(4) 완료
리뷰 마지막 페이지
#_review_paging > a:nth-child(5)
https://cr.shopping.naver.com/adcr.nhn?x=bBTj67Ba%2BzMN8lXjQ3KkD%2F%2F%2F%2Fw%3D%3Dsk0es6GxRC44IulkwnoQ5M3hURWgkvEclVHEVLHhtY%2FWT5c0h3fMq5Lgja49KBeY2EKXes9BpmSsOTdoC4yHpNhP4U8tqKz%2BHkqhuKTj1OB6GeJ%2B93nTTrrU%2FPUNRK1UR61k45S%2FknuIuxGYzTyZzPwRt2twIl0PFqGTdKBTp7ZXDpmNu2Rs3tEPrV0ZTODsbtTVoPK0bnhgZAVEMq%2FNgnJK2bckh09gtxpLhV5kqbdAUB2rokDitWE7orcTpopd%2BM6uQ0%2FEuxe9Kx7BiV%2BSF2Vw2EzC1R7jC%2F43hDec%2FYLmc%2BzySEcyja6yI2wIa7nRnca58VC9yAmQyC04nezu%2B%2BWz8zbWMtJTFaAZuTYJYj51YO4Am6plsaXHSO650j1yFP2tfKoQ0NQPyiad34AQ6K9%2FdVvwtcOUlbNrGLU3RUnr0%2Bj95ttUXspOo1OFcx5xUFY0AqSJOZMeLCVtoa0X5yhBWPSmnGo10wxV7%2BzhZBNA5bRq1Ayw9NfVYIT11SaP6mc2cg3xqNUkKGEb1xg81nXcGJ3yzdya85llEHAo24K053dOWh1dJcXPfhyAeBRR7tocvNpKTxeylx%2FnM6s6cUvY1KhbJLmHDGWNpM7gYeA9Vaj8KvtfLLJVSo8yN4E9MegPqOOK7kO9ce%2BrDs1mvZeHRJ52Zz7d2kaQJTcsJRdnwdW4fhQAuSM3Snu5RVyv4%2F3Shv47XvNlFK6QDh5tKut6xdFIb1X%2BCEtxF

In [39]:
prdInfo_list

[['뉴나리프바운서', '4.8', '뉴나', '뉴나', '2013.05.', '85', '248,000'],
 ['스펙트라젖병소독기', '4.9', '유진메디케어', '스펙트라', '2017.09.', '225', '218,000']]

In [63]:
keyword = input("검색할 키워드를 입력하세요: ")
url = get_keywordUrl(keyword)
url_list = get_detailUrl(url)

print('각 상품에 대한 url 수집 완료')

# 상품 정보
prdInfo_list = list()
# 리뷰 정보
rvw_list = list()

filename = keyword + '_' + str(date.today()) + '.csv'
product_csv_file = open(filename, "a", encoding = 'utf-8-sig', newline= '')
wr = csv.writer(product_csv_file)
wr.writerow(['상품 이름', '평점', '제조사', '브랜드명', '등록일', '찜(좋아요) 갯수', '가격'])

for url in url_list:
    temp_prdInfo_list = get_prdInfo(url)
    prdInfo_list.append(temp_prdInfo_list)

    temp_rvw_list = get_prdRvw(url)
    rvw_list.append(temp_rvw_list)
    print(url, '하나의 상품 정보 및 이에 대한 리뷰 수집 완료')
print('전체 상품 정보 및 이에 대한 리뷰 수집 완료')

# 상품정보 파일 작성 (list)
for prdInfo in prdInfo_list:
    wr.writerow(prdInfo)
print('상품정보 파일 저장완료')
product_csv_file.close()


# 리뷰정보 파일 작성
for idx, reviews in enumerate(rvw_list):
    filename = keyword + '_' + str(prdInfo_list[idx][0]) + str(date.today()) + '.csv'
    review_csv_file = open(filename, "a", encoding = 'utf-8-sig', newline= '')
    wr = csv.writer(review_csv_file)
    wr.writerow(['평점', '브랜드명', '고객 ID', '등록일', '제목', '본문', '이미지 url'])    

    for review in reviews:
        wr.writerow(review)
    review_csv_file.close()
    print(prdInfo_list[idx][0], '리뷰정보 파일 저장완료')


검색할 키워드를 입력하세요: 닭가슴살 도시락
각 상품에 대한 url 수집 완료
1 ~ 10page
#_review_paging > a:nth-child(2)완료
#_review_paging > a:nth-child(3)완료
#_review_paging > a:nth-child(4)완료
#_review_paging > a:nth-child(5)완료
#_review_paging > a:nth-child(6)완료
#_review_paging > a:nth-child(7)완료
#_review_paging > a:nth-child(8)완료
#_review_paging > a:nth-child(9)완료
리뷰 마지막 페이지
#_review_paging > a:nth-child(10)
11page ~ 
#_review_paging > a:nth-child(4) 완료
#_review_paging > a:nth-child(5) 완료
#_review_paging > a:nth-child(6) 완료
#_review_paging > a:nth-child(7) 완료
#_review_paging > a:nth-child(8) 완료
#_review_paging > a:nth-child(9) 완료
리뷰 마지막 페이지
#_review_paging > a:nth-child(10)
https://cr.shopping.naver.com/adcr.nhn?x=jn4NaGekk%2FacbcZcUmffRf%2F%2F%2Fw%3D%3Ds%2F47Yy0TKOY%2FtW6bYC4IxKnzQzGXcGyb70bn%2BEY1Xju5URgVbdo%2F4PXOwP%2BCkYSb%2BIvsF232LdCw5Yjw4baJ6SWS7UnsKZtB7wGu55neDqAJeJCWYoDA0IA3jyca%2FZGcnlmdpSXbJCA9T1WrNYG7nKP9JqkRUjRYTcmohgBteequAbCY81f975eS5nfR25FBitmbKT6fiobaO7w0fEN2NzhTtRLWgUgHVCqjbH6kiokWC%2B7e6juVAlxPTHfq

https://cr.shopping.naver.com/adcr.nhn?x=zYEp1dObMFvmlu26T47GtP%2F%2F%2Fw%3D%3DshCuHTZGZW5GGMAay5LXG1bKJJ%2BBFjEkr9GyudmYpal9URgVbdo%2F4PXOwP%2BCkYSb%2BIvsF232LdCw5Yjw4baJ6SUC%2FgcDDNL7fdif%2BCNOspudeJCWYoDA0IA3jyca%2FZGcnlmdpSXbJCA9T1WrNYG7nKP9JqkRUjRYTcmohgBteequAbCY81f975eS5nfR25FBitmbKT6fiobaO7w0fEN2NzmJGvUKRtLjcaptWMenqhqbWCsIExFV0diS6GEcDQHNxM6uQ0%2FEuxe9Kx7BiV%2BSF2Vw2EzC1R7jC%2F43hDec%2FYLmc%2BzySEcyja6yI2wIa7nRnca58VC9yAmQyC04nezu%2B%2BWz8zbWMtJTFaAZuTYJYj51YO4Am6plsaXHSO650j1yFP2tfKoQ0NQPyiad34AQ6K9%2FdVvwtcOUlbNrGLU3RUnpojwX7Z2LH3aIuhmKGl2Xuai83EVbMDvzV06wNeZeBjZ%2Bca9tTOMn220t4i9iLe3tYK%2FNUN442dEkfO%2FtkLjKcnD38MxfZLVnMXa3E7TvClDP%2F6iiIn7QzI%2BXIqwB6FMvMcgwNALQ4lDpEqf1%2FGEUUhamfVl2hWFu5rqbLeGo%2FLLXjoDrdxPCKm7cKEATA5to3WOvmXRjNqlwVnzGXNyutm5qLRT9MoMXtcpS0wZvXShCQTUjyvDTLnnCzaXjR4TFxtplKINvTBNAF4tVLGvVEiA3%2FU2AzguC84AnoS7Lq%2BWPDzS7X5yyi%2BZ2TjDsG%2FKbLx6iz4l7dQDpxY8mnJnnW5Hv1T%2FoDPeAT2ZD%2BWCfnsQ%3D%3D&nvMid=23231616490&catId=50006199 하나의 상품 정보 및 이에 대한 리뷰 수집 완료
1 ~ 10p

#_review_paging > a:nth-child(2)완료
리뷰 마지막 페이지
#_review_paging > a:nth-child(3)
11page ~ 
#_review_paging > a:nth-child(4) 완료
리뷰 마지막 페이지
#_review_paging > a:nth-child(5)
https://cr.shopping.naver.com/adcr.nhn?x=%2F6NeB1%2FxIoNyDkdEyjjv4%2F%2F%2F%2Fw%3D%3Ds%2BLeQuSyENL3Nr4eT%2FKFQpeqLIZUYTpR47YVeZ4aGRtTwTKoXjM94GI4SFQFG0OJqfiLZUrJg2%2FgUUcH2C820DOmARDoqNRD5KsmvKwSfxyuwkx1ctIZSLUjBFsRFpgKsQXPPHgBv0ftnayBBT9j15medW%2BMyCIcyNsWGbi%2FX%2FUA%2Fa18qhDQ1A%2FKJp3fgBDorJDuXZdQuo4LCHwzw3HG6PHDnrfZgH9emf9Xiotgw397YY9juXog%2BipzWS7NHDmc36xD57hFIbQb%2FklB900yo%2BloDjcQBDY1F%2BKzX2ZnBz%2BArdF411kcE99Rr6HERw6m9t5CiGfC2CLp%2BEuKfGzFA7wF7isUVsp3M6Ct9mT6vB4JWEXeXwSaubPu0qUaxQfPPa%2BpHa6ewb0%2FDpcGez9%2B2Mmaf7oVVn%2BbTDsruUdhAA7ti46CBp10XuGqMo4iYcU%2B7OiSHso0SSRvAEEo5QU4QSoPc8hmJjVjFIDxGbc%2FxawzZMgYtkLkLrXnMU%2Bz0EQl3IjbxAMh%2B10%2BjmxnveZkXVNflZ4wXf0rC9wPy3zCyJ2eIBP2K5aPfkKD37wcC%2BNiMV3%2Bk4WWnH1Pc9NYjYXmwFUzdzXcnFBtYUEiS6IRpgAMpmcOxb%2F2KdO0B9s5isqguQAj3hXUxQYYLRbaC82hV2wBIJKxdQP%2BRJkKWr6lzaHWIDf9TYDO

https://cr.shopping.naver.com/adcr.nhn?x=SP%2B3jBBx%2B4xvurZ%2BTC3emP%2F%2F%2Fw%3D%3DsECFbpFRuOXzJNyHiTjoD0zp8f8bhTyMiPmbZn0QoqsiT5c0h3fMq5Lgja49KBeY2EKXes9BpmSsOTdoC4yHpNryN1FKzReBdVd6b6vOKht7m6HT9CGVmA81JgoOXalh9Uw93dHFQn%2BmyLcIXSbLPKuMxnSPGnOR55n7IhMgIb%2FnDpmNu2Rs3tEPrV0ZTODsbtTVoPK0bnhgZAVEMq%2FNgnLp%2FhWo7X1eaWnUQ38A2%2Frn9qfrbG%2F9ekdyXVpWdbIhbM6uQ0%2FEuxe9Kx7BiV%2BSF2Vw2EzC1R7jC%2F43hDec%2FYLmc%2BzySEcyja6yI2wIa7nRnca58VC9yAmQyC04nezu%2B%2BWz8zbWMtJTFaAZuTYJYj51YO4Am6plsaXHSO650j1yFP2tfKoQ0NQPyiad34AQ6K9%2FdVvwtcOUlbNrGLU3RUnrtG76m74tQ888vWJWT3jDlFY0AqSJOZMeLCVtoa0X5yrmMBK1pivmLwJ%2B5FYHrUjFBbAPJtJ6RPIukLNKgrZFJmc2cg3xqNUkKGEb1xg81nVkbephCbL7JOdURGygKzO4RDiXs0jtXtb%2FqsxtwFC%2BuIOWLmVF0VnF5BPnUwURF%2BSxL0am7%2FoAChKHn5nyltneyJf1269SZ1RulkvTeG7WfSjQdzDJtHVc4mFFoxuqsdqtV%2Byq6qYTLD1qa54pUgQqIDf9TYDOC4LzgCehLsur5MYcUfyjJWD%2BW%2FFNBn4tbI8S6X2yeDsQRFWM1PuIeop1Xc50eBITwugSNPH3Ok09sVpGw61BSdoQRvEGgfFlc5w%3D%3D&nvMid=21255052849&catId=50001878 하나의 상품 정보 및 이에 대한 리뷰 수집 완료
1 ~ 10page
#_

11page ~ 
#_review_paging > a:nth-child(4) 완료
리뷰 마지막 페이지
#_review_paging > a:nth-child(5)
https://cr.shopping.naver.com/adcr.nhn?x=HiwhtjdH7GvX8IoA19F%2Fjv%2F%2F%2Fw%3D%3DsyKTVSWbhvjI2Obko1Omays4RiBU%2F%2FjPSorCHjrYFRfGT5c0h3fMq5Lgja49KBeY2EKXes9BpmSsOTdoC4yHpNn3uN8zIsttOc1CQFgd4XX7m6HT9CGVmA81JgoOXalh9Uw93dHFQn%2BmyLcIXSbLPKuMxnSPGnOR55n7IhMgIb%2FnDpmNu2Rs3tEPrV0ZTODsbtTVoPK0bnhgZAVEMq%2FNgnB7kzrtwmWT5RdWcJWX6k0CyufIgL2Pb3oG4avsYUvW6M6uQ0%2FEuxe9Kx7BiV%2BSF2Vw2EzC1R7jC%2F43hDec%2FYLmc%2BzySEcyja6yI2wIa7nRnca58VC9yAmQyC04nezu%2B%2BWz8zbWMtJTFaAZuTYJYj51YO4Am6plsaXHSO650j1yFP2tfKoQ0NQPyiad34AQ6K9%2FdVvwtcOUlbNrGLU3RUnpn8XOSwnrsbu17zxdZxUCoFY0AqSJOZMeLCVtoa0X5yrmMBK1pivmLwJ%2B5FYHrUjG5BwOFh4KKrPmzKinicddHmc2cg3xqNUkKGEb1xg81nYZ1Xf5RN3YKmpS6Fax1XcmoDZtPVPWr4qyf9VT49VH0fod0RdRC6IjgWjcaCxu%2FNEZn%2BKMntXYz8KHBgoGUF%2B%2FmtF3ymVaCaru0icbs7AQwtdR%2F1eQ4BUzKIEK3XoO4uVO24jCYH6jBan6U%2Fj8MwpgNauEcRynSn%2F8orOK7nc90mTk2Bc%2BYsnHM8WVQSCpJ3EJZC%2BWewjbuHpmFdRZ9E3BWhZ3SMHTcLlpircdRzUgq1JIslBmdxaTx7x

https://cr.shopping.naver.com/adcr.nhn?x=D8EMBqzP8DZ3VfH%2FvnUPKv%2F%2F%2Fw%3D%3DsYQbpmGy8NDM39bYDgHeumZfsW9lyZ9%2B7SYLzNJJ8JI2T5c0h3fMq5Lgja49KBeY2EKXes9BpmSsOTdoC4yHpNi2KdISNRKjX6UJJhFPMbbzm6HT9CGVmA81JgoOXalh9Uw93dHFQn%2BmyLcIXSbLPKuMxnSPGnOR55n7IhMgIb%2FnDpmNu2Rs3tEPrV0ZTODsbtTVoPK0bnhgZAVEMq%2FNgnIwh5cXP6e%2Bsf5bpOD7%2BNPKomrE7i5UntK5uVaeXsjb0M6uQ0%2FEuxe9Kx7BiV%2BSF2Vw2EzC1R7jC%2F43hDec%2FYLmc%2BzySEcyja6yI2wIa7nRnca58VC9yAmQyC04nezu%2B%2BWz8zbWMtJTFaAZuTYJYj51YO4Am6plsaXHSO650j1yFP2tfKoQ0NQPyiad34AQ6K9%2FdVvwtcOUlbNrGLU3RUnp0g3LdPIP1Zkilnyn0bu6FFY0AqSJOZMeLCVtoa0X5yrmMBK1pivmLwJ%2B5FYHrUjEUh5E8uR2h9li4H8ZKVH6lmc2cg3xqNUkKGEb1xg81nUx0cPPjs%2FDf%2BBakjqSpHIVJQbsBPMZN0y%2FuIM%2FoMmf5DnqD6xzrSqDHk7bs3%2BZKwDeY5T%2BegczEjsOn2OpbpCddWr8USHpPdANVzsaJDvAssakRy0zX%2FSY4P86157U1yWjH7Q2m7wiFMLarSC1RZ7dcGUuYSqgJBuzNaqr8UeaGYiIdiWIflXMNWtLeEnV%2BMLGP8cdKYADVhoGB3S10kc1f%2BG144%2Fl3nAaapiv62oPKp2AgzVvx3l%2FWE8bDH579Ox0cjwVm1BUYVknE7IEMpz4%3D&nvMid=22137976581&catId=50001878 하나의 상품 정보 및 이에 대한 

#_review_paging > a:nth-child(3)
11page ~ 
#_review_paging > a:nth-child(4) 완료
리뷰 마지막 페이지
#_review_paging > a:nth-child(5)
https://cr.shopping.naver.com/adcr.nhn?x=MJUjJYEnWa259o3aG%2BA2cf%2F%2F%2Fw%3D%3DsU0kIUF8ABlTQK%2FzHu6VAsLwaPUALYkpW10I3AepnYjFURgVbdo%2F4PXOwP%2BCkYSb%2BIvsF232LdCw5Yjw4baJ6SW9oBLIUxxNQLRWx9bwtCDxeJCWYoDA0IA3jyca%2FZGcnlmdpSXbJCA9T1WrNYG7nKP9JqkRUjRYTcmohgBteequAbCY81f975eS5nfR25FBipriVTNUFNXA4oBwrJtENpHglRPJVBVhnYuI4OiXSxf7oZTvuVAEDESyKD6Y%2Fg81P6xD57hFIbQb%2FklB900yo%2BloDjcQBDY1F%2BKzX2ZnBz%2BArdF411kcE99Rr6HERw6m9t5CiGfC2CLp%2BEuKfGzFA7wF7isUVsp3M6Ct9mT6vB4JWEXeXwSaubPu0qUaxQfPPa%2BpHa6ewb0%2FDpcGez9%2B2Mmaf7oVVn%2BbTDsruUdhAA7uezGl6jIfndxW4KQDaq85x1EL7PBh6vo%2Bzr1Db8r4ssVb%2F0ZeebLeRbodpyOZhJGi%2BkvHv7xoj6vFX1X83eaX%2FBHS7iYdd4BTrgASNQmm8sB1C8JSNVK48y11SY5Dhyqm2%2Fnt1LHXcSACFki56Wcjhl5LTNY%2FCpTSjns6%2BiwPIaVoRB50aA7H59k9ooSxsI4AQsc4Q6%2FrNXSDhA5Fd2NC8QUppKJ8cl3OpENS70beQcekfgABKVHfiohLC25VW%2BrrSFWuvoNBKwsFA2IsGyJy5MHQ6tCOoT4YnWr7V0Gn7dU%2FAJcUmJF2ix%2Feizcr